In [2]:
import tensorflow as tf
import numpy as np

# To make VPINS we need to:  
#                                                          
#   1) Define a basis for V_n                                   
max_n = 10

def basisdif_fn(i,max_n):
    basisdif_functions=[]
    for i in range(1,max_n+1):
        basisdif_fi = lambda x, i=i: np.pi*n*tf.cos(n*tf.constant(np.pi, dtype=tf.float32)*x)
        basisdif_functions.append(basisdif_fi)


def basis_fn(i,max_n):
    basis_functions=[]
    for i in range(1,max_n+1):
        basis_fi = lambda x, i=i: tf.sin(i*tf.constant(np.pi, dtype=tf.float32)*x)
        basis_functions.append(basis_fi)        
 
    
                                                                                          
# 2) Define a class for gaussian integration

class Gaussint:
    def __init__(self, function, n_points = 2):
        self.function = function
        self.n_points = n_points
        self.nodes, self.weights = np.polynomial.legendre.leggauss(n_points)
        self.nodes_tf = tf.constant(self.nodes, dtype = tf.float32)
        self.weights_tf = tf.constant(self.weights, dtype = tf.float32)

    def integrate(self, a, b):
        x_mapped = ((b-a)/2.0)*self.nodes_tf + (b+a)/2.0
        function_values = self.function(x_mapped)
        integral = (b-a)/2.0 * tf.reduce_sum(self.weights_tf * function_values)
        return integral


#Assemble the loss

class loss(tf.keras.layers.Layer):

    def __init__(self, U_model, N_points,f_source, N_test **kwargs):
        super(loss,self).__init__()
        self.U_model = U_model
        self.N_points = N_points
        self.f_source = f_source
        self.N_test = N_test

#Compute the derivative of the neural network U_nn
    def  call(self,inputs):
        
        #Get a sample of points in the domain for the integration. 

        S = tf.experimental.numpy.linspace(0.,np.pi,num=self.N_points, dtype='float32')
       
        with tf.GradientTape(persistent=True) as t1:       
            t1.watch(S)
            ueval = self.U_model(S)
            u_x = t1.gradient(ueval,S)
              
        #Compute a vector of the source function evaluated in the sample points & 
        F = self.f_source(S)
        G = [tf.sin(i*np.pi*S) for i in range(self.N_test)]
        Integral_source = tf.einsum('i,ki->k',F,G)
        H = [i*np.pi*tf.cos(i*np.pi*S) for i in range(self.N_test)]
        Integral_Unn = tf.einsum('i,ki->k',u_x,H)
        
        return tf.reduce_sum((Integral_source - Integral_Unn)**2)



#Calculate the integrals for l(basis_n) and Δu_nn and Δbasis_n
basis_functions = basis_functions
integrals = []
for i in range (1,max_n):
    initialize = Gaussint(, n_points=3)

a, b = 0, np.pi
result = integrator.integrate(a, b)

#Subtract and take the square for r(u_nn,basis_n)


#Add r(u_nn,basis_n) from 1 to max_n



        #Assemble the residual loss & the loss for the collocation points
        f_source = self.f_source(S)
        loss_out = tf.math.reduce_mean((u_xx - f_source)**2) + self.U_model(np.array([0.]))**2 + self.U_model(np.array([np.pi]))**2 
        tf.print(loss_out)
        return loss_out




In [16]:
class param_basis:
    def __init__(self, max_n):
        self.max_n =int(max_n)
        self.pi = np.pi
    def get_basis_functions(self):
        basis_functions = []
        for n in range(1,self.max_n + 1):
            basis_fn = lambda x, n=n: tf.sin(n*tf.constant(self.pi, dtype=tf.float32) * x) 
            basis_functions.append(basis_fn)
        return basis_functions    
    
max_n = 10.0
basis_generator = param_basis(max_n)
basis_functions = basis_generator.get_basis_functions()
print(basis_functions)

[<function param_basis.get_basis_functions.<locals>.<lambda> at 0x71e1f3ae5c60>, <function param_basis.get_basis_functions.<locals>.<lambda> at 0x71e1f3ae5a80>, <function param_basis.get_basis_functions.<locals>.<lambda> at 0x71e1f3ae5d00>, <function param_basis.get_basis_functions.<locals>.<lambda> at 0x71e1f3ae5da0>, <function param_basis.get_basis_functions.<locals>.<lambda> at 0x71e1f3ae5e40>, <function param_basis.get_basis_functions.<locals>.<lambda> at 0x71e1f3ae5ee0>, <function param_basis.get_basis_functions.<locals>.<lambda> at 0x71e1f3ae5f80>, <function param_basis.get_basis_functions.<locals>.<lambda> at 0x71e1f3ae6020>, <function param_basis.get_basis_functions.<locals>.<lambda> at 0x71e1f3ae60c0>, <function param_basis.get_basis_functions.<locals>.<lambda> at 0x71e1f3ae6160>]
